# 🧾 Personal Expense Tracker — Interactive (Widget-based)

**Features included**
- Add / Edit / Delete expenses via interactive widgets  
- List all expenses in a table view  
- Summaries by Category and Month  
- Visualizations (Bar: category totals, Line: monthly trend)  
- Persistent storage in `expenses.json` with sample data auto-generation  
- Clean markdown sections and project-style organization


## 📚 Imports and Setup

In [5]:
import json
import os
from typing import List, Dict, Any, Optional
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import ipywidgets as widgets
from IPython.display import display, clear_output
DATA_FILE = "expenses.json"


## 💾 Data Loading & Saving (JSON)

In [6]:
def load_expenses(filename: str = DATA_FILE) -> List[Dict[str, Any]]:
    if not os.path.exists(filename):
        return []
    try:
        with open(filename, 'r', encoding='utf-8') as f:
            data = json.load(f)
        if isinstance(data, list):
            return data
    except Exception:
        # If file corrupted or unreadable, return empty list
        return []
    return []

def save_expenses(expenses: List[Dict[str, Any]], filename: str = DATA_FILE) -> None:
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(expenses, f, indent=4, ensure_ascii=False)

def ensure_sample_data():
    expenses = load_expenses()
    if not expenses:
        sample = [
            {"Date": "2025-10-01", "Category": "Food", "Amount": 200.0, "Description": "Breakfast"},
            {"Date": "2025-10-03", "Category": "Transport", "Amount": 120.0, "Description": "Bus ticket"},
            {"Date": "2025-10-05", "Category": "Shopping", "Amount": 500.0, "Description": "Groceries"},
            {"Date": "2025-09-28", "Category": "Bills", "Amount": 1500.0, "Description": "Electricity bill"}
        ]
        save_expenses(sample)
        return sample
    return expenses

# initialize file if missing
ensure_sample_data()


[{'amount': 50.0, 'category': 'ghh', 'date': '2025-10-16'}]

## 🧰 DataFrame helper

In [7]:
def load_as_dataframe() -> pd.DataFrame:
    expenses = load_expenses()
    df = pd.DataFrame(expenses)
    if df.empty:
        return df
    # Ensure correct dtypes
    if 'Date' in df.columns:
        df['Date'] = pd.to_datetime(df['Date'])
        df['Month'] = df['Date'].dt.to_period('M')
    if 'Amount' in df.columns:
        df['Amount'] = pd.to_numeric(df['Amount'], errors='coerce').fillna(0.0)
    return df

# quick check
df = load_as_dataframe()
print(f"Loaded {len(df)} records from {DATA_FILE}")


Loaded 1 records from expenses.json


## 🔧 Core functions (add, edit, delete, list, summary, visuals)

In [11]:
def add_expense(date: str, category: str, amount: float, description: str) -> None:
    expenses = load_expenses()
    new = {"Date": date, "Category": category, "Amount": float(amount), "Description": description}
    expenses.append(new)
    save_expenses(expenses)

def list_expenses() -> pd.DataFrame:
    return load_as_dataframe()

def find_expense_index_by_rowid(rowid: int) -> Optional[int]:
    expenses = load_expenses()
    if 0 <= rowid < len(expenses):
        return rowid
    return None

def edit_expense(index: int, date: str, category: str, amount: float, description: str) -> bool:
    expenses = load_expenses()
    if 0 <= index < len(expenses):
        expenses[index] = {"Date": date, "Category": category, "Amount": float(amount), "Description": description}
        save_expenses(expenses)
        return True
    return False

def delete_expense(index: int) -> bool:
    expenses = load_expenses()
    if 0 <= index < len(expenses):
        expenses.pop(index)
        save_expenses(expenses)
        return True
    return False

def view_summary() -> Dict[str, pd.DataFrame]:
    df = load_as_dataframe()
    if df.empty:
        return {"by_category": pd.DataFrame(), "by_month": pd.DataFrame()}
    by_category = df.groupby('Category', as_index=False)['Amount'].sum().sort_values('Amount', ascending=False)
    by_month = df.groupby('Month', as_index=False)['Amount'].sum().sort_values('Month')
    return {"by_category": by_category, "by_month": by_month}

def show_visual_summary(output_area=None):
    df = load_as_dataframe()
    if df.empty:
        if output_area:
            with output_area:
                clear_output()
                print("No data to visualize.")
        return
    summary = view_summary()
    by_category = summary['by_category']
    by_month = summary['by_month']
    # Create plots (each its own figure)
    fig1, ax1 = plt.subplots(figsize=(8,4))
    ax1.bar(by_category['Category'], by_category['Amount'])
    ax1.set_title('Total Expenses by Category')
    ax1.set_xlabel('Category')
    ax1.set_ylabel('Total Amount (₹)')
    fig1.tight_layout()
    fig2, ax2 = plt.subplots(figsize=(8,4))
    # by_month['Month'] might be Period objects; convert to string for plotting
    ax2.plot(by_month['Month'].astype(str), by_month['Amount'], marker='o')
    ax2.set_title('Monthly Expense Trend')
    ax2.set_xlabel('Month')
    ax2.set_ylabel('Total Amount (₹)')
    fig2.tight_layout()
    if output_area:
        with output_area:
            clear_output()
            display(fig1)
            display(fig2)
    else:
        display(fig1)
        display(fig2)


## 🧭 Interactive Widget UI (Menu-driven)

In [12]:
# Output area for messages and tables
out = widgets.Output(layout={'border': '1px solid lightgray'})

# --- Add Expense Widgets ---
date_in = widgets.Text(value=datetime.today().strftime('%Y-%m-%d'), description='Date:')
cat_in = widgets.Text(value='Food', description='Category:')
amt_in = widgets.FloatText(value=0.0, description='Amount:')
desc_in = widgets.Text(value='', description='Description:')
add_btn = widgets.Button(description='Add Expense', button_style='success')

def on_add_clicked(b):
    try:
        add_expense(date_in.value, cat_in.value, amt_in.value, desc_in.value)
        with out:
            clear_output()
            print('✅ Expense added.')
            display(list_expenses().tail(5))
    except Exception as e:
        with out:
            clear_output()
            print('Error adding expense:', e)

add_btn.on_click(on_add_clicked)
add_box = widgets.VBox([widgets.Label('Add a new expense'), date_in, cat_in, amt_in, desc_in, add_btn])

# --- List / Refresh ---
refresh_btn = widgets.Button(description='Refresh List')
list_box_out = widgets.Output()

def on_refresh(b):
    with list_box_out:
        clear_output()
        df = list_expenses()
        if df.empty:
            print('No expenses found.')
        else:
            display(df.reset_index().rename(columns={'index':'RowID'}))

refresh_btn.on_click(on_refresh)
list_box = widgets.VBox([widgets.Label('All expenses (press Refresh)'), refresh_btn, list_box_out])

# --- Edit Expense Widgets ---
edit_index = widgets.IntText(value=0, description='RowID:')
edit_date = widgets.Text(value=datetime.today().strftime('%Y-%m-%d'), description='Date:')
edit_cat = widgets.Text(value='Food', description='Category:')
edit_amt = widgets.FloatText(value=0.0, description='Amount:')
edit_desc = widgets.Text(value='', description='Description:')
edit_btn = widgets.Button(description='Edit Expense', button_style='warning')

def on_edit(b):
    idx = edit_index.value
    success = edit_expense(idx, edit_date.value, edit_cat.value, edit_amt.value, edit_desc.value)
    with out:
        clear_output()
        if success:
            print(f'✅ Expense at RowID {idx} updated.')
            display(list_expenses().reset_index().rename(columns={'index':'RowID'}).loc[[idx]])
        else:
            print('❌ Invalid RowID or edit failed.')

edit_btn.on_click(on_edit)
edit_box = widgets.VBox([widgets.Label('Edit an expense (provide RowID)'), edit_index, edit_date, edit_cat, edit_amt, edit_desc, edit_btn])

# --- Delete Expense Widgets ---
del_index = widgets.IntText(value=0, description='RowID:')
del_btn = widgets.Button(description='Delete Expense', button_style='danger')

def on_delete(b):
    idx = del_index.value
    success = delete_expense(idx)
    with out:
        clear_output()
        if success:
            print(f'🗑️ Deleted expense at RowID {idx}.')
            display(list_expenses().reset_index().rename(columns={'index':'RowID'}).head(10))
        else:
            print('❌ Invalid RowID or delete failed.')

del_btn.on_click(on_delete)
del_box = widgets.VBox([widgets.Label('Delete an expense (provide RowID)'), del_index, del_btn])

# --- Summary & Visuals ---
summary_btn = widgets.Button(description='Show Summary')
visual_btn = widgets.Button(description='Show Visual Summary')

summary_out = widgets.Output()
visual_out = widgets.Output()

def on_summary(b):
    with summary_out:
        clear_output()
        s = view_summary()
        print('### By Category')
        display(s['by_category'])
        print('\n### By Month')
        display(s['by_month'])

def on_visual(b):
    show_visual_summary(output_area=visual_out)

summary_btn.on_click(on_summary)
visual_btn.on_click(on_visual)

summary_box = widgets.HBox([summary_btn, visual_btn])
summary_display = widgets.VBox([summary_box, summary_out, visual_out])

# --- Layout ---
tab = widgets.Tab()
tab.children = [add_box, list_box, edit_box, del_box, summary_display, out]
tab.set_title(0, 'Add')
tab.set_title(1, 'List')
tab.set_title(2, 'Edit')
tab.set_title(3, 'Delete')
tab.set_title(4, 'Summary & Visuals')
tab.set_title(5, 'Messages')

display(tab)


## ✅ Conclusion

This interactive notebook provides a complete personal expense tracker with persistent JSON storage, editable records, summaries, and visualizations. It's ready for submission as a mini project.